In [18]:
import os, sys
sys.path.insert(1,'/home/doga/GithUBuntU/keyhandshapediscovery')
import helperFuncs as funcH
import numpy as np
import pandas as pd

In [2]:
n = 21 # 21 samples 
klusters = np.asarray([0,0,1,1,2,2,3,3,0,0,4,4,0,5,0,5,1,2,3,1,5],dtype=int)
labels   = np.asarray([0,0,0,1,0,0,1,1,0,1,2,2,0,2,2,2,0,2,1,0,0],dtype=int)

In [3]:
_confMat, kluster2Classes, kr_pdf, weightedPurity, cnmxh_perc = funcH.countPredictionsForConfusionMat(labels, klusters, labelNames=["c0","c1","c2"], centroid_info_pdf=None, verbose=0)

In [4]:
print(kluster2Classes)

[[0, 0, 6, 4], [1, 0, 4, 3], [2, 0, 3, 2], [3, 1, 3, 3], [4, 2, 2, 2], [5, 2, 3, 2]]


In [5]:
k2c = pd.DataFrame(kluster2Classes,columns=["klust","mappedClass","elementCnt","correctMapCount"],index=["k0","k1","k2","k3","k4","k5"])
print(k2c)

    klust  mappedClass  elementCnt  correctMapCount
k0      0            0           6                4
k1      1            0           4                3
k2      2            0           3                2
k3      3            1           3                3
k4      4            2           2                2
k5      5            2           3                2


In [6]:
print(kr_pdf)

  kID mappedClass  #of  N     %purity
3  k3          c1    3  3  100.000000
4  k4          c2    2  2  100.000000
1  k1          c0    3  4   75.000000
0  k0          c0    4  6   66.666667
2  k2          c0    2  3   66.666667
5  k5          c2    2  3   66.666667


### Lets create a sample distance matrix for the above assignments

In [7]:
dm = np.zeros((6,6))
print(dm)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [8]:
dm[0,1]= 5;dm[0,2]=7;dm[0,3]=15;dm[0,4]=22;dm[0,5]=26
dm[1,2]= 8;dm[1,3]=21;dm[1,4]=16;dm[1,5]=17
dm[2,3]= 6;dm[2,4]=28;dm[2,5]=23
dm[3,4]=25;dm[3,5]=22
dm[4,5]=4
for i in range(6):
    for j in range(6):
        if j>i:
            dm[j,i] = dm[i,j]

In [9]:
print(dm)

[[ 0.  5.  7. 15. 22. 26.]
 [ 5.  0.  8. 21. 16. 17.]
 [ 7.  8.  0.  6. 28. 23.]
 [15. 21.  6.  0. 25. 22.]
 [22. 16. 28. 25.  0.  4.]
 [26. 17. 23. 22.  4.  0.]]


In [10]:
dm_pddf = pd.DataFrame(np.array(dm))
print(dm_pddf)

      0     1     2     3     4     5
0   0.0   5.0   7.0  15.0  22.0  26.0
1   5.0   0.0   8.0  21.0  16.0  17.0
2   7.0   8.0   0.0   6.0  28.0  23.0
3  15.0  21.0   6.0   0.0  25.0  22.0
4  22.0  16.0  28.0  25.0   0.0   4.0
5  26.0  17.0  23.0  22.0   4.0   0.0


### what next
I need to use the dm or dm_pddf

I will create a function that takes:

dm, a clusterID(rowID), combinedClusterSets

there are k sets and all sets have a distance between each other


In [11]:
def create_dist_vec(X):
    kcnt = len(X)
    row_cnt = int(kcnt*(kcnt-1)*.5)
    dv = np.array(np.zeros((row_cnt,4)), dtype=object)
    print(dv.shape)
    k = 0
    for i in range(0,kcnt):
        for j in range(i+1,kcnt):
            dv[k,0] = "{}".format(i)
            dv[k,1] = "{}".format(j)
            dv[k,2] = X[i,j]
            dv[k,3] = X[i,j]
            k=k+1
    return dv
dv = create_dist_vec(dm)

(15, 4)


In [12]:
print(dv)

[['0' '1' 5.0 5.0]
 ['0' '2' 7.0 7.0]
 ['0' '3' 15.0 15.0]
 ['0' '4' 22.0 22.0]
 ['0' '5' 26.0 26.0]
 ['1' '2' 8.0 8.0]
 ['1' '3' 21.0 21.0]
 ['1' '4' 16.0 16.0]
 ['1' '5' 17.0 17.0]
 ['2' '3' 6.0 6.0]
 ['2' '4' 28.0 28.0]
 ['2' '5' 23.0 23.0]
 ['3' '4' 25.0 25.0]
 ['3' '5' 22.0 22.0]
 ['4' '5' 4.0 4.0]]


In [13]:
minMul=2.0
maxMul=1.0
def sort_dv(dist_vec, minMul=1.0, maxMul=1.0):
    min_d_vec = dist_vec[:,2]
    max_d_vec = dist_vec[:,3]
    d_sort_vec = (minMul*min_d_vec/(max(min_d_vec))+maxMul*max_d_vec/(max(max_d_vec)))/(minMul+maxMul)
    idx = np.argsort(d_sort_vec)
    arshp = (dist_vec.shape[0],5)
    dvs = np.array(np.zeros(arshp), dtype=object)
    dvs[:,:-1] = dist_vec[idx,:]
    dvs[:,4] = d_sort_vec[idx]
    return dvs
dvs = sort_dv(dv, minMul=minMul, maxMul=maxMul)
print(dvs)

[['4' '5' 4.0 4.0 0.14285714285714285]
 ['0' '1' 5.0 5.0 0.17857142857142858]
 ['2' '3' 6.0 6.0 0.21428571428571427]
 ['0' '2' 7.0 7.0 0.25]
 ['1' '2' 8.0 8.0 0.2857142857142857]
 ['0' '3' 15.0 15.0 0.5357142857142857]
 ['1' '4' 16.0 16.0 0.5714285714285714]
 ['1' '5' 17.0 17.0 0.6071428571428571]
 ['1' '3' 21.0 21.0 0.75]
 ['0' '4' 22.0 22.0 0.7857142857142857]
 ['3' '5' 22.0 22.0 0.7857142857142857]
 ['2' '5' 23.0 23.0 0.8214285714285715]
 ['3' '4' 25.0 25.0 0.8928571428571429]
 ['0' '5' 26.0 26.0 0.9285714285714285]
 ['2' '4' 28.0 28.0 1.0]]


In [14]:
groups_dict = {}
first_kluster = dvs[0,0]
groups_dict["G01"] = first_kluster
for i in range(dvs.shape[0]):
    if dvs[i,0]==first_kluster:
        dvs[i,0]="G01"
    if dvs[i,1]==first_kluster:
        dvs[i,1]="G01"
decision_row = dvs[0,:]

### first decision 

1- get the closest samples [4-5 are closest]

2- annotate one of them as G1.

3- put the other "in the spot" - let the annotator annotate it with G2 or G1
   tell the min and max distance the have to each other
   
4- once the decision is made we either have G1 and G2 or only G1

In [15]:
len(groups_dict)

1

In [16]:
print(groups_dict)
print(decision_row)

{'G01': '4'}
['G01' '5' 4.0 4.0 0.14285714285714285]


In [17]:
def add_to_group(groups_dict, group_name, klusterID, dvs):
    # now add klusterID to group with name group_name in the groups_dict
    if group_name is None or group_name=="":
        group_name = str(len(groups_dict)+1).zfill(2)
        groups_dict[group_name] = ""
    old_group_elements = groups_dict[group_name]
    print("old_group_elements:<{}>".format(old_group_elements))
    
    new_group_elements = old_group_elements.split(',')
    print("new_group_elements1:<{}>".format(new_group_elements))
    
    new_group_elements = new_group_elements.append([str(klusterID)])
    print(type(new_group_elements))
    print("new_group_elements2:<{}>".format(new_group_elements))
    
    new_group_elements = ','.join(new_group_elements)
    print("new_group_elements3:<{}>".format(new_group_elements))

add_to_group(groups_dict, group_name="G01", klusterID=decision_row[1], dvs=dvs)

old_group_elements:<4>
new_group_elements1:<['4']>
<class 'NoneType'>
new_group_elements2:<None>


TypeError: can only join an iterable